# Autoencoders
---

We will explore the use of autoencoders for automatic feature engineering. The idea is to automatically learn a set of features from raw data that can be useful in supervised learning tasks such as in computer vision and insurance.

## Computer Vision
---

We will use the MNIST dataset for this purpose where the raw data is a 2 dimensional tensor of pixel intensities per image. The image is our unit of analysis: We will predict the probability of each class for each image. This is a multiclass classification task and we will use the one against all AUROC score and accuracy score to assess model performance on the test fold.

## Insurance
---

We will use a dataset from the R package "insuranceData" where the raw data is a 2 dimensional tensor of historical policy level information per policy-period combination. The policy-period combination is our unit of analysis: We will predict the probability of loss for each policy-period combination. This is a binary class classification task and we will use the AUROC score and accuracy score to assess model performance.

In [1]:
# Author: Hamaad Musharaf Shah.

import os
import math
import sys
import importlib

import numpy as np

import pandas as pd

from sklearn import linear_model
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer
from sklearn.metrics import roc_auc_score

from scipy.stats import norm

import keras
from keras import backend as bkend
from keras.datasets import cifar10, mnist
from keras.layers import Dense, BatchNormalization, Dropout, Flatten, convolutional, pooling
from keras import metrics

from autoencoders_keras.get_session import get_session
import keras.backend.tensorflow_backend as KTF
KTF.set_session(get_session(gpu_fraction=0.75, allow_soft_placement=True, log_device_placement=False))

import tensorflow as tf
from tensorflow.python.client import device_lib

from plotnine import *

import matplotlib.pyplot as plt

from autoencoders_keras.vanilla_autoencoder import VanillaAutoencoder
from autoencoders_keras.convolutional_autoencoder import ConvolutionalAutoencoder
from autoencoders_keras.convolutional2D_autoencoder import Convolutional2DAutoencoder
from autoencoders_keras.seq2seq_autoencoder import Seq2SeqAutoencoder
from autoencoders_keras.variational_autoencoder import VariationalAutoencoder

%matplotlib inline

os.environ["KERAS_BACKEND"] = "tensorflow"
importlib.reload(bkend)

print(device_lib.list_local_devices())

mnist = mnist.load_data()
(X_train, y_train), (X_test, y_test) = mnist
X_train = np.reshape(X_train, [X_train.shape[0], X_train.shape[1] * X_train.shape[1]])
X_test = np.reshape(X_test, [X_test.shape[0], X_test.shape[1] * X_test.shape[1]])
y_train = y_train.ravel()
y_test = y_test.ravel()
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255.0
X_test /= 255.0

Using TensorFlow backend.
/Users/samson/anaconda3/envs/autoencoders/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
Using TensorFlow backend.


[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9822157098076685420
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 1810374656
locality {
  bus_id: 1
}
incarnation: 13296979930656387060
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:c2:00.0"
]


## Scikit-learn
---

We will use the Python machine learning library scikit-learn for data transformation and the classification task. Note that we will code the autoencoders as scikit-learn transformers such that they can be readily used by scikit-learn pipelines.

In [2]:
scaler_classifier = MinMaxScaler(feature_range=(0.0, 1.0))
logistic = linear_model.LogisticRegression(random_state=666)
lb = LabelBinarizer()
lb = lb.fit(y_train.reshape(y_train.shape[0], 1))

## MNIST: No Autoencoders
---

We run the MNIST dataset without using an autoencoder. The 2 dimensional tensor of pixel intensities per image for MNIST images are of dimension $\mathbb{R}^{28 \times 28}$. We reshape them as a 1 dimensional tensor of dimension $\mathbb{R}^{784}$ per image. Therefore we have 784, i.e., $28 \times 28 = 784$, features for this supervised learning task per image.

In [ ]:
pipe_base = Pipeline(steps=[("scaler_classifier", scaler_classifier),
                            ("classifier", logistic)])
pipe_base = pipe_base.fit(X_train, y_train)

auroc_base = roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)),
                           pipe_base.predict_proba(X_test), 
                           average="weighted")

acc_base = pipe_base.score(X_test, y_test)

print("The AUROC score for the MNIST classification task without autoencoders: %.6f%%." % (auroc_base * 100))
print("The accuracy score for the MNIST classification task without autoencoders: %.6f%%." % (acc_base * 100))

## MNIST: Vanilla Autoencoders
---

An autoencoder is an unsupervised learning technique where the objective is to learn a set of features that can be used to reconstruct the input data.

Our input data is $X \in \mathbb{R}^{N \times 784}$. An encoder function $E$ maps this to a set of $K$ features such that $E: \mathbb{R}^{N \times 784} \rightarrow \mathbb{R}^{N \times K}$. A decoder function $D$ uses the set of $K$ features to reconstruct the input data such that $D: \mathbb{R}^{N \times K} \rightarrow \mathbb{R}^{N \times 784}$. 
    
Lets denote the reconstructed data as $\tilde{X} = D(E(X))$. The goal is to learn the encoding and decoding functions such that we minimize the difference between the input data and the reconstructed data. An example for an objective function for this task can be the Mean Squared Error (MSE) such that $\frac{1}{N}||\tilde{X} - X||^{2}_{2}$. 
    
We learn the encoding and decoding functions by minimizing the MSE using the parameters that define the encoding and decoding functions: The gradient of the MSE with respect to the parameters are calculated using the chain rule, i.e., backpropagation, and used to update the parameters via an optimization algorithm such as Stochastic Gradient Descent (SGD). 

Lets assume we have a single layer autoencoder using the Exponential Linear Unit (ELU) activation function, batch normalization, dropout and the Adaptive Moment (Adam) optimization algorithm. $B$ is the batch size, $K$ is the number of features.

* **Exponential Linear Unit:** The activation function is smooth everywhere and avoids the vanishing gradient problem as the output takes on negative values when the input is negative. $\alpha$ is taken to be $1.0$.

    \begin{align}
    H_{\alpha}(z) &= 
    \begin{cases}
    \alpha\left(\exp(z) - 1\right) &\text{ if } z < 0 \\
    z &\text{ if } z \geq 0
    \end{cases} \\
    \frac{dH_{\alpha}(z)}{dz} &= 
    \begin{cases}
    \alpha\left(\exp(z)\right) &\text{ if } z < 0 \\
    1 &\text{ if } z \geq 0
    \end{cases} 
    \end{align}


* **Batch Normalization:** The idea is to transform the inputs into a hidden layer's activation functions. We standardize or normalize first using the mean and variance parameters on a per feature basis and then learn a set of scaling and shifting parameters on a per feature basis that transforms the data. The following equations describe this layer succintly: The parameters we learn in this layer are $\left(\mu_{j}, \sigma_{j}^2, \beta_{j}, \gamma_{j}\right) \hspace{0.1cm} \forall j \in \{1, \dots, K\}$.

    \begin{align}
    \mu_{j} &= \frac{1}{B} \sum_{i=1}^{B} X_{i,j} \hspace{0.1cm} &\forall j \in \{1, \dots, K\} \\
    \sigma_{j}^2 &= \frac{1}{B} \sum_{i=1}^{B} \left(X_{i,j} - \mu_{j}\right)^2 \hspace{0.1cm} &\forall j \in \{1, \dots, K\} \\
    \hat{X}_{:,j} &= \frac{X_{:,j} - \mu_{j}}{\sqrt{\sigma_{j}^2 + \epsilon}} \hspace{0.1cm} &\forall j \in \{1, \dots, K\} \\
    Z_{:,j} &= \gamma_{j}\hat{X}_{:,j} + \beta_{j} \hspace{0.1cm} &\forall j \in \{1, \dots, K\} \\
    \end{align}


* **Dropout:** This regularization technique simply drops the outputs from input and hidden units with a certain probability say $50\%$. 


* **Adam Optimization Algorithm:** This adaptive algorithm combines ideas from the Momentum and RMSProp optimization algorithms. The goal is to have some memory of past gradients which can guide future parameters updates. The following equations for the algorithm succintly describe this method assuming $\theta$ is our set of parameters to be learnt and $\eta$ is the learning rate.

    \begin{align}
    m &\leftarrow \beta_{1}m + \left[\left(1 - \beta_{1}\right)\left(\nabla_{\theta}\text{MSE}\right)\right] \\
    s &\leftarrow \beta_{2}s + \left[\left(1 - \beta_{2}\right)\left(\nabla_{\theta}\text{MSE} \otimes \nabla_{\theta}\text{MSE} \right)\right] \\
    \theta &\leftarrow \theta - \eta m \oslash \sqrt{s + \epsilon}
    \end{align}

In [3]:
autoencoder = VanillaAutoencoder(n_feat=X_train.shape[1],
                                 n_epoch=50,
                                 batch_size=100,
                                 encoder_layers=3,
                                 decoder_layers=3,
                                 n_hidden_units=1000,
                                 encoding_dim=500,
                                 denoising=None)

print(autoencoder.autoencoder.summary())

pipe_autoencoder = Pipeline(steps=[("autoencoder", autoencoder),
                                   ("scaler_classifier", scaler_classifier),
                                   ("classifier", logistic)])

pipe_autoencoder = pipe_autoencoder.fit(X_train, y_train)

auroc_autoencoder = roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
                                  pipe_autoencoder.predict_proba(X_test), 
                                  average="weighted")
    
acc_autoencoder = pipe_autoencoder.score(X_test, y_test)

print("The AUROC score for the MNIST classification task with an autoencoder: %.6f%%." % (auroc_autoencoder * 100))
print("The accuracy score for the MNIST classification task with an autoencoder: %.6f%%." % (acc_autoencoder * 100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 784)               3136      
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              785000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 1000)              4000      
_________________________________________________________________
dense_2 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000)              0         
__________

Train on 42000 samples, validate on 18000 samples
Epoch 1/50
42000/42000 [==============================] - 10s 244us/step - loss: 0.0478 - val_loss: 0.0177
Epoch 2/50
42000/42000 [==============================] - 8s 194us/step - loss: 0.0212 - val_loss: 0.0150
Epoch 3/50
42000/42000 [==============================] - 8s 195us/step - loss: 0.0192 - val_loss: 0.0133
Epoch 4/50
42000/42000 [==============================] - 8s 202us/step - loss: 0.0181 - val_loss: 0.0124
Epoch 5/50
42000/42000 [==============================] - 9s 207us/step - loss: 0.0174 - val_loss: 0.0117
Epoch 6/50
42000/42000 [==============================] - 9s 203us/step - loss: 0.0168 - val_loss: 0.0113
Epoch 7/50
42000/42000 [==============================] - 8s 201us/step - loss: 0.0165 - val_loss: 0.0110
Epoch 8/50
42000/42000 [==============================] - 8s 198us/step - loss: 0.0161 - val_loss: 0.0106
Epoch 9/50
42000/42000 [==============================] - 9s 202us/step - loss: 0.0158 - val_loss: 0.

42000/42000 [==============================] - 8s 196us/step - loss: 0.0130 - val_loss: 0.0087
Epoch 34/50
42000/42000 [==============================] - 8s 196us/step - loss: 0.0129 - val_loss: 0.0086
Epoch 35/50
42000/42000 [==============================] - 8s 198us/step - loss: 0.0129 - val_loss: 0.0083
Epoch 36/50
42000/42000 [==============================] - 8s 195us/step - loss: 0.0128 - val_loss: 0.0085
Epoch 37/50
42000/42000 [==============================] - 8s 195us/step - loss: 0.0127 - val_loss: 0.0083
Epoch 38/50
42000/42000 [==============================] - 8s 196us/step - loss: 0.0127 - val_loss: 0.0083
Epoch 39/50
42000/42000 [==============================] - 8s 195us/step - loss: 0.0126 - val_loss: 0.0083
Epoch 40/50
42000/42000 [==============================] - 8s 195us/step - loss: 0.0125 - val_loss: 0.0083
Epoch 41/50
42000/42000 [==============================] - 8s 195us/step - loss: 0.0125 - val_loss: 0.0081
Epoch 42/50
42000/42000 [========================

## MNIST: Denoising Autoencoders
---

The idea here is to add some noise to the data and try to learn a set of robust features that can reconstruct the non-noisy data from the noisy data. The MSE objective functions is as follows, $\frac{1}{N}||D(E(X + \epsilon)) - X||^{2}_{2}$, where $\epsilon$ is some noise term.

In [ ]:
noise = 0.10 * np.reshape(np.random.uniform(low=0.0, 
                                            high=1.0, 
                                            size=X_train.shape[0] * X_train.shape[1]), 
                          [X_train.shape[0], X_train.shape[1]])

denoising_autoencoder = VanillaAutoencoder(n_feat=X_train.shape[1],
                                           n_epoch=50,
                                           batch_size=100,
                                           encoder_layers=3,
                                           decoder_layers=3,
                                           n_hidden_units=1000,
                                           encoding_dim=500,
                                           denoising=noise)

print(denoising_autoencoder.autoencoder.summary())

pipe_denoising_autoencoder = Pipeline(steps=[("autoencoder", denoising_autoencoder),
                                             ("scaler_classifier", scaler_classifier),
                                             ("classifier", logistic)])

pipe_denoising_autoencoder = pipe_denoising_autoencoder.fit(X_train, y_train)

auroc_denoising_autoencoder = roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
                                            pipe_denoising_autoencoder.predict_proba(X_test), 
                                            average="weighted")

acc_denoising_autoencoder = pipe_denoising_autoencoder.score(X_test, y_test)

print("The AUROC score for the MNIST classification task with a denoising autoencoder: %.6f%%." % (auroc_denoising_autoencoder * 100))
print("The accuracy score for the MNIST classification task with a denoising autoencoder: %.6f%%." % (acc_denoising_autoencoder * 100))

## MNIST: 1 Dimensional Convolutional Autoencoders
---

In [ ]:
convolutional_autoencoder = ConvolutionalAutoencoder(input_shape=(int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5))),
                                                     n_epoch=50,
                                                     batch_size=100,
                                                     encoder_layers=3,
                                                     decoder_layers=3,
                                                     filters=100,
                                                     kernel_size=5,
                                                     strides=1,
                                                     pool_size=4,
                                                     encoding_dim=100,
                                                     denoising=None)

print(convolutional_autoencoder.autoencoder.summary())

pipe_convolutional_autoencoder = Pipeline(steps=[("autoencoder", convolutional_autoencoder),
                                                 ("scaler_classifier", scaler_classifier),
                                                 ("classifier", logistic)])

pipe_convolutional_autoencoder = pipe_convolutional_autoencoder.fit(np.reshape(X_train, [X_train.shape[0], int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5))]), 
                                                                    y_train)

auroc_convolutional_autoencoder = roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
                                                pipe_convolutional_autoencoder.predict_proba(np.reshape(X_test, [X_test.shape[0], int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5))])),
                                                average="weighted")

acc_convolutional_autoencoder = pipe_convolutional_autoencoder.score(np.reshape(X_test, [X_test.shape[0], int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5))]), y_test)

print("The AUROC score for the MNIST classification task with a 1 dimensional convolutional autoencoder: %.6f%%." % (auroc_convolutional_autoencoder * 100))
print("The accuracy score for the MNIST classification task with a 1 dimensional convolutional autoencoder: %.6f%%." % (acc_convolutional_autoencoder * 100))

## MNIST: Sequence to Sequence Autoencoders
---

In [ ]:
seq2seq_autoencoder = Seq2SeqAutoencoder(input_shape=(int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5))),
                                         n_epoch=50,
                                         batch_size=100,
                                         encoder_layers=3,
                                         decoder_layers=3,
                                         n_hidden_units=100,
                                         encoding_dim=100,
                                         stateful=False,
                                         denoising=None)

print(seq2seq_autoencoder.autoencoder.summary())

pipe_seq2seq_autoencoder = Pipeline(steps=[("autoencoder", seq2seq_autoencoder),
                                           ("scaler_classifier", scaler_classifier),
                                           ("classifier", logistic)])

pipe_seq2seq_autoencoder = pipe_seq2seq_autoencoder.fit(np.reshape(X_train, [X_train.shape[0], int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5))]),
                                                        y_train)

auroc_seq2seq_autoencoder = roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
                                          pipe_seq2seq_autoencoder.predict_proba(np.reshape(X_test, [X_test.shape[0], int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5))])),
                                          average="weighted")

acc_seq2seq_autoencoder = pipe_seq2seq_autoencoder.score(np.reshape(X_test, [X_test.shape[0], int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5))]), y_test)

print("The AUROC score for the MNIST classification task with a sequence to sequence autoencoder: %.6f%%." % (auroc_seq2seq_autoencoder * 100))
print("The accuracy score for the MNIST classification task with a sequence to sequence autoencoder: %.6f%%." % (acc_seq2seq_autoencoder * 100))

## MNIST: Variational Autoencoders
---

In [ ]:
encoding_dim = 500

variational_autoencoder = VariationalAutoencoder(n_feat=X_train.shape[1],
                                                 n_epoch=50,
                                                 batch_size=100,
                                                 encoder_layers=3,
                                                 decoder_layers=3,
                                                 n_hidden_units=500,
                                                 encoding_dim=encoding_dim,
                                                 denoising=None)

print(variational_autoencoder.autoencoder.summary())

pipe_variational_autoencoder = Pipeline(steps=[("autoencoder", variational_autoencoder),
                                               ("scaler_classifier", scaler_classifier),
                                               ("classifier", logistic)])

pipe_variational_autoencoder = pipe_variational_autoencoder.fit(X_train, y_train)

auroc_variational_autoencoder = roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
                                              pipe_variational_autoencoder.predict_proba(X_test), 
                                              average="weighted")

acc_variational_autoencoder = pipe_variational_autoencoder.score(X_test, y_test)

print("The AUROC score for the MNIST classification task with a sequence to variational autoencoder: %.6f%%." % (auroc_variational_autoencoder * 100))
print("The accuracy score for the MNIST classification task with a sequence to variational autoencoder: %.6f%%." % (acc_variational_autoencoder * 100))

if encoding_dim == 2:
    test_encoded_df = pd.DataFrame(pipe_variational_autoencoder.named_steps["autoencoder"].encoder.predict(X_test))
    test_encoded_df["Target"] = y_test
    test_encoded_df.columns.values[0:2] = ["Encoding_1", "Encoding_2"]

    scaler_plot = MinMaxScaler(feature_range=(0.25, 0.75))
    scaler_plot = scaler_plot.fit(test_encoded_df[["Encoding_1", "Encoding_2"]])
    test_encoded_df[["Encoding_1", "Encoding_2"]] = scaler_plot.transform(test_encoded_df[["Encoding_1", "Encoding_2"]])

    cluster_plot = ggplot(test_encoded_df) + \
    geom_point(aes(x="Encoding_1", 
                   y="Encoding_2", 
                   fill="factor(Target)"),
               size=1,
               color = "black") + \
    xlab("Encoding dimension 1") + \
    ylab("Encoding dimension 2") + \
    ggtitle("Variational autoencoder with 2-dimensional encoding") + \
    theme_matplotlib()
    print(cluster_plot)

    n = 15
    digit_size = 28
    figure = np.zeros((digit_size * n, digit_size * n))
    grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
    grid_y = norm.ppf(np.linspace(0.05, 0.95, n))

    for i, xi in enumerate(grid_x):
        for j, yi in enumerate(grid_y):
            z_sample = np.array([[xi, yi]])
            x_decoded = pipe_variational_autoencoder.named_steps["autoencoder"].generator.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size)
            figure[i * digit_size: (i + 1) * digit_size, j * digit_size: (j + 1) * digit_size] = digit

    plt.figure(figsize=(10, 10))
    plt.imshow(figure, cmap="Greys_r")
    plt.title("Variational autoencoder with 2-dimensional encoding\nGenerating new images")
    plt.xlabel("")
    plt.ylabel("")
    plt.show()

## MNIST: 2 Dimensional Convolutional Autoencoders
---

In [ ]:
convolutional2D_autoencoder = Convolutional2DAutoencoder(input_shape=(int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5)), 1),
                                                         n_epoch=5,
                                                         batch_size=100,
                                                         encoder_layers=3,
                                                         decoder_layers=3,
                                                         filters=25,
                                                         kernel_size=5,
                                                         strides=1,
                                                         pool_size=4,
                                                         denoising=None)

print(convolutional2D_autoencoder.autoencoder.summary())

pipe_convolutional2D_autoencoder = Pipeline(steps=[("autoencoder", convolutional2D_autoencoder),
                                                   ("scaler_classifier", scaler_classifier),
                                                   ("classifier", logistic)])

pipe_convolutional2D_autoencoder = pipe_convolutional2D_autoencoder.fit(np.reshape(X_train, [X_train.shape[0], int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5)), 1]),
                                                                        y_train)

auroc_convolutional2D_autoencoder = roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
                                                  pipe_convolutional2D_autoencoder.predict_proba(np.reshape(X_test, [X_test.shape[0], int(math.pow(X_train.shape[1], 0.5)), int(math.pow(X_train.shape[1], 0.5)), 1])),
                                                  average="weighted")

acc_convolutional2D_autoencoder = pipe_convolutional2D_autoencoder.score(np.reshape(X_test, [X_test.shape[0], int(math.pow(X_test.shape[1], 0.5)), int(math.pow(X_test.shape[1], 0.5)), 1]), y_test)

print("The AUROC score for the MNIST classification task with a sequence to 2 dimensional convolutional autoencoder: %.6f%%." % (auroc_convolutional2D_autoencoder * 100))
print("The accuracy score for the MNIST classification task with a sequence to 2 dimensional convolutional autoencoder: %.6f%%." % (acc_convolutional2D_autoencoder * 100))

## References
---

1. Goodfellow, I., Bengio, Y. and Courville A. (2016). Deep Learning (MIT Press).
2. Geron, A. (2017). Hands-On Machine Learning with Scikit-Learn & Tensorflow (O'Reilly).
3. http://scikit-learn.org/stable/#
4. https://towardsdatascience.com/learning-rate-schedules-and-adaptive-learning-rate-methods-for-deep-learning-2c8f433990d1
5. https://stackoverflow.com/questions/42177658/how-to-switch-backend-with-keras-from-tensorflow-to-theano
6. https://blog.keras.io/building-autoencoders-in-keras.html
7. https://keras.io

## Appendix: 2 Dimensional Convolutional Autoencoders for CIFAR
---

In [31]:
cifar = cifar10.load_data()
(X_train, y_train), (X_test, y_test) = cifar
y_train = y_train.ravel()
y_test = y_test.ravel()
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255.0
X_test /= 255.0

convolutional2D_autoencoder = Convolutional2DAutoencoder(input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]),
                                                         n_epoch=50,
                                                         batch_size=100,
                                                         encoder_layers=3,
                                                         decoder_layers=3,
                                                         filters=50,
                                                         kernel_size=4,
                                                         strides=1,
                                                         pool_size=4,
                                                         denoising=None)

print(convolutional2D_autoencoder.autoencoder.summary())

pipe_convolutional2D_autoencoder = Pipeline(steps=[("autoencoder", convolutional2D_autoencoder)])

pipe_convolutional2D_autoencoder = pipe_convolutional2D_autoencoder.fit(X_train)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
batch_normalization_51 (Batc (None, 32, 32, 3)         12        
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 50)        2450      
_________________________________________________________________
dropout_41 (Dropout)         (None, 32, 32, 50)        0         
_________________________________________________________________
batch_normalization_52 (Batc (None, 32, 32, 50)        200       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 32, 32, 50)        40050     
_________________________________________________________________
dropout_42 (Dropout)         (None, 32, 32, 50)        0         
__________

Train on 35000 samples, validate on 15000 samples
Epoch 1/50
35000/35000 [==============================] - 45s 1ms/step - loss: 0.0138 - val_loss: 0.0146
Epoch 2/50
35000/35000 [==============================] - 42s 1ms/step - loss: 0.0067 - val_loss: 0.0056
Epoch 3/50
35000/35000 [==============================] - 42s 1ms/step - loss: 0.0058 - val_loss: 0.0047
Epoch 4/50
35000/35000 [==============================] - 42s 1ms/step - loss: 0.0053 - val_loss: 0.0044
Epoch 5/50
35000/35000 [==============================] - 42s 1ms/step - loss: 0.0050 - val_loss: 0.0034
Epoch 6/50
35000/35000 [==============================] - 42s 1ms/step - loss: 0.0047 - val_loss: 0.0032
Epoch 7/50
35000/35000 [==============================] - 42s 1ms/step - loss: 0.0046 - val_loss: 0.0036
Epoch 8/50
35000/35000 [==============================] - 42s 1ms/step - loss: 0.0044 - val_loss: 0.0029
Epoch 9/50
35000/35000 [==============================] - 42s 1ms/step - loss: 0.0044 - val_loss: 0.0030
Epoch

## Appendix: Transfer Learning for CIFAR
---

In [33]:
num_classes = 10

with tf.device("/gpu:0"):
    autoencoder = pipe_convolutional2D_autoencoder.named_steps["autoencoder"].autoencoder

    encoder_at_layer = 10
    for layer in autoencoder.layers[0:encoder_at_layer]:
        layer.trainable = False

    classifier = autoencoder.layers[encoder_at_layer - 1].output
    classifier = Flatten()(classifier)
    
    classifier = BatchNormalization()(classifier)
    classifier = Dense(units=1000, activation="elu")(classifier)
    classifier = Dropout(rate=0.5)(classifier)

    classifier = BatchNormalization()(classifier)
    classifier = Dense(units=500, activation="elu")(classifier)
    classifier = Dropout(rate=0.5)(classifier)

    classifier = BatchNormalization()(classifier)
    classifier = Dense(units=200, activation="elu")(classifier)
    classifier = Dropout(rate=0.5)(classifier)

    classifier = BatchNormalization()(classifier)
    classifier = Dense(units=100, activation="elu")(classifier)
    classifier = Dropout(rate=0.5)(classifier)

    classifier = BatchNormalization()(classifier)
    classifier = Dense(units=100, activation="elu")(classifier)

    classifier = Dense(units=num_classes, activation="softmax")(classifier)

    model_final = Model(autoencoder.input, classifier)
    model_final.compile(loss="categorical_crossentropy", 
                        optimizer=keras.optimizers.Adam(), 
                        metrics=[metrics.categorical_accuracy])

    model_final.summary()

    model_final.fit(X_train, keras.utils.to_categorical(y_train, num_classes),
                    validation_split=0.1,
                    epochs=50,
                    batch_size=100,
                    shuffle=True,
                    verbose=1)

print(roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
                    model_final.predict(X_test), 
                    average="weighted"))

print(model_final.evaluate(X_test, keras.utils.to_categorical(y_test, num_classes)))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 3072)              0         
_________________________________________________________________
batch_normalization_63 (Batc (None, 3072)              12288     
_________________________________________________________________
dense_66 (Dense)             (None, 1000)              3073000   
_________________________________________________________________
dropout_50 (Dropout)         (None, 1000)              0         
_________________________________________________________________
batch_normalization_64 (Batc (None, 1000)              4000      
_________________________________________________________________
dense_67 (Dense)             (None, 500)               500500    
__________

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 10s 215us/step - loss: 1.9232 - categorical_accuracy: 0.3029 - val_loss: 1.6623 - val_categorical_accuracy: 0.3980
Epoch 2/50
45000/45000 [==============================] - 7s 151us/step - loss: 1.7258 - categorical_accuracy: 0.3778 - val_loss: 1.5691 - val_categorical_accuracy: 0.4266
Epoch 3/50
45000/45000 [==============================] - 7s 150us/step - loss: 1.6466 - categorical_accuracy: 0.4065 - val_loss: 1.4905 - val_categorical_accuracy: 0.4694
Epoch 4/50
45000/45000 [==============================] - 7s 149us/step - loss: 1.5814 - categorical_accuracy: 0.4319 - val_loss: 1.4349 - val_categorical_accuracy: 0.4850
Epoch 5/50
45000/45000 [==============================] - 7s 151us/step - loss: 1.5326 - categorical_accuracy: 0.4548 - val_loss: 1.4031 - val_categorical_accuracy: 0.4976
Epoch 6/50
45000/45000 [==============================] - 7s 149us/step - loss: 1.5016 - c

45000/45000 [==============================] - 7s 148us/step - loss: 1.0384 - categorical_accuracy: 0.6420 - val_loss: 1.1932 - val_categorical_accuracy: 0.5828
Epoch 40/50
45000/45000 [==============================] - 7s 148us/step - loss: 1.0228 - categorical_accuracy: 0.6465 - val_loss: 1.1788 - val_categorical_accuracy: 0.5898
Epoch 41/50
45000/45000 [==============================] - 7s 148us/step - loss: 1.0198 - categorical_accuracy: 0.6495 - val_loss: 1.1730 - val_categorical_accuracy: 0.5922
Epoch 42/50
45000/45000 [==============================] - 7s 148us/step - loss: 1.0168 - categorical_accuracy: 0.6491 - val_loss: 1.1708 - val_categorical_accuracy: 0.5870
Epoch 43/50
45000/45000 [==============================] - 7s 149us/step - loss: 1.0064 - categorical_accuracy: 0.6536 - val_loss: 1.1709 - val_categorical_accuracy: 0.5956
Epoch 44/50
45000/45000 [==============================] - 7s 148us/step - loss: 1.0036 - categorical_accuracy: 0.6538 - val_loss: 1.1833 - val_cat

In [36]:
num_classes = 10

with tf.device("/gpu:0"):
    autoencoder = pipe_convolutional2D_autoencoder.named_steps["autoencoder"].autoencoder

    encoder_at_layer = 10
    for layer in autoencoder.layers[0:encoder_at_layer]:
        layer.trainable = False

    classifier = autoencoder.layers[encoder_at_layer - 1].output
    
    classifier = convolutional.UpSampling2D(4)(classifier)
    
    classifier = BatchNormalization()(classifier)
    classifier = convolutional.Conv2D(filters=50, kernel_size=4, strides=1, activation="elu", padding="same")(classifier)
    classifier = Dropout(rate=0.5)(classifier)

    classifier = BatchNormalization()(classifier)
    classifier = convolutional.Conv2D(filters=50, kernel_size=4, strides=1, activation="elu", padding="same")(classifier)
    classifier = Dropout(rate=0.5)(classifier)

    classifier = BatchNormalization()(classifier)
    classifier = convolutional.Conv2D(filters=50, kernel_size=4, strides=1, activation="elu", padding="same")(classifier)
    classifier = pooling.MaxPooling2D(4, padding="same")(classifier)
    
    classifier = Flatten()(classifier)
    
    classifier = BatchNormalization()(classifier)
    classifier = Dense(units=1000, activation="elu")(classifier)
    classifier = Dropout(rate=0.5)(classifier)

    classifier = BatchNormalization()(classifier)
    classifier = Dense(units=500, activation="elu")(classifier)
    classifier = Dropout(rate=0.5)(classifier)

    classifier = BatchNormalization()(classifier)
    classifier = Dense(units=200, activation="elu")(classifier)
    classifier = Dropout(rate=0.5)(classifier)

    classifier = BatchNormalization()(classifier)
    classifier = Dense(units=100, activation="elu")(classifier)
    classifier = Dropout(rate=0.5)(classifier)

    classifier = BatchNormalization()(classifier)
    classifier = Dense(units=100, activation="elu")(classifier)

    classifier = Dense(units=num_classes, activation="softmax")(classifier)

    model_final = Model(autoencoder.input, classifier)
    model_final.compile(loss="categorical_crossentropy", 
                        optimizer=keras.optimizers.Adam(), 
                        metrics=[metrics.categorical_accuracy])

    model_final.summary()

    model_final.fit(X_train, keras.utils.to_categorical(y_train, num_classes),
                    validation_split=0.1,
                    epochs=50,
                    batch_size=100,
                    shuffle=True,
                    verbose=1)

print(roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
                    model_final.predict(X_test), 
                    average="weighted"))

print(model_final.evaluate(X_test, keras.utils.to_categorical(y_test, num_classes)))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
batch_normalization_51 (Batc (None, 32, 32, 3)         12        
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 50)        2450      
_________________________________________________________________
dropout_41 (Dropout)         (None, 32, 32, 50)        0         
_________________________________________________________________
batch_normalization_52 (Batc (None, 32, 32, 50)        200       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 32, 32, 50)        40050     
_________________________________________________________________
dropout_42 (Dropout)         (None, 32, 32, 50)        0         
__________

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 43s 951us/step - loss: 1.5236 - categorical_accuracy: 0.4471 - val_loss: 2.1126 - val_categorical_accuracy: 0.3942
Epoch 2/50
45000/45000 [==============================] - 38s 850us/step - loss: 1.1780 - categorical_accuracy: 0.5855 - val_loss: 1.5633 - val_categorical_accuracy: 0.5196
Epoch 3/50
45000/45000 [==============================] - 38s 851us/step - loss: 1.0472 - categorical_accuracy: 0.6369 - val_loss: 1.2697 - val_categorical_accuracy: 0.5956
Epoch 4/50
45000/45000 [==============================] - 38s 852us/step - loss: 0.9684 - categorical_accuracy: 0.6679 - val_loss: 1.2342 - val_categorical_accuracy: 0.6078
Epoch 5/50
45000/45000 [==============================] - 38s 853us/step - loss: 0.8967 - categorical_accuracy: 0.6964 - val_loss: 0.8746 - val_categorical_accuracy: 0.7076
Epoch 6/50
45000/45000 [==============================] - 39s 856us/step - loss: 0.843

Epoch 7/50
45000/45000 [==============================] - 38s 854us/step - loss: 0.7970 - categorical_accuracy: 0.7313 - val_loss: 0.8683 - val_categorical_accuracy: 0.7166
Epoch 8/50
45000/45000 [==============================] - 38s 854us/step - loss: 0.7425 - categorical_accuracy: 0.7510 - val_loss: 0.7487 - val_categorical_accuracy: 0.7442
Epoch 9/50
45000/45000 [==============================] - 38s 855us/step - loss: 0.7068 - categorical_accuracy: 0.7638 - val_loss: 0.8148 - val_categorical_accuracy: 0.7320
Epoch 10/50
45000/45000 [==============================] - 38s 854us/step - loss: 0.6707 - categorical_accuracy: 0.7758 - val_loss: 0.8119 - val_categorical_accuracy: 0.7330
Epoch 11/50
45000/45000 [==============================] - 38s 854us/step - loss: 0.6462 - categorical_accuracy: 0.7824 - val_loss: 0.7766 - val_categorical_accuracy: 0.7498
Epoch 12/50
45000/45000 [==============================] - 38s 855us/step - loss: 0.6115 - categorical_accuracy: 0.7972 - val_loss: 0

In [ ]:
num_classes = 10

with tf.device("/gpu:0"):
    autoencoder = pipe_convolutional2D_autoencoder.named_steps["autoencoder"].autoencoder

    classifier = autoencoder.layers[0].output
    classifier = Flatten()(classifier)
    
    classifier = BatchNormalization()(classifier)
    classifier = Dense(units=1000, activation="elu")(classifier)
    classifier = Dropout(rate=0.5)(classifier)

    classifier = BatchNormalization()(classifier)
    classifier = Dense(units=500, activation="elu")(classifier)
    classifier = Dropout(rate=0.5)(classifier)

    classifier = BatchNormalization()(classifier)
    classifier = Dense(units=200, activation="elu")(classifier)
    classifier = Dropout(rate=0.5)(classifier)

    classifier = BatchNormalization()(classifier)
    classifier = Dense(units=100, activation="elu")(classifier)
    classifier = Dropout(rate=0.5)(classifier)

    classifier = BatchNormalization()(classifier)
    classifier = Dense(units=100, activation="elu")(classifier)

    classifier = Dense(units=num_classes, activation="softmax")(classifier)

    model_final = Model(autoencoder.input, classifier)
    model_final.compile(loss="categorical_crossentropy", 
                        optimizer=keras.optimizers.Adam(), 
                        metrics=[metrics.categorical_accuracy])

    model_final.summary()

    model_final.fit(X_train, keras.utils.to_categorical(y_train, num_classes),
                    validation_split=0.1,
                    epochs=50,
                    batch_size=100,
                    shuffle=True,
                    verbose=1)

print(roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
                    model_final.predict(X_test), 
                    average="weighted"))

print(model_final.evaluate(X_test, keras.utils.to_categorical(y_test, num_classes)))

In [ ]:
num_classes = 10

with tf.device("/gpu:0"):
    autoencoder = pipe_convolutional2D_autoencoder.named_steps["autoencoder"].autoencoder

    classifier = autoencoder.layers[0].output
    
    classifier = BatchNormalization()(classifier)
    classifier = convolutional.Conv2D(filters=50, kernel_size=4, strides=1, activation="elu", padding="same")(classifier)
    classifier = Dropout(rate=0.5)(classifier)

    classifier = BatchNormalization()(classifier)
    classifier = convolutional.Conv2D(filters=50, kernel_size=4, strides=1, activation="elu", padding="same")(classifier)
    classifier = Dropout(rate=0.5)(classifier)

    classifier = BatchNormalization()(classifier)
    classifier = convolutional.Conv2D(filters=50, kernel_size=4, strides=1, activation="elu", padding="same")(classifier)
    classifier = pooling.MaxPooling2D(4, padding="same")(classifier)
    
    classifier = Flatten()(classifier)
    
    classifier = BatchNormalization()(classifier)
    classifier = Dense(units=1000, activation="elu")(classifier)
    classifier = Dropout(rate=0.5)(classifier)

    classifier = BatchNormalization()(classifier)
    classifier = Dense(units=500, activation="elu")(classifier)
    classifier = Dropout(rate=0.5)(classifier)

    classifier = BatchNormalization()(classifier)
    classifier = Dense(units=200, activation="elu")(classifier)
    classifier = Dropout(rate=0.5)(classifier)

    classifier = BatchNormalization()(classifier)
    classifier = Dense(units=100, activation="elu")(classifier)
    classifier = Dropout(rate=0.5)(classifier)

    classifier = BatchNormalization()(classifier)
    classifier = Dense(units=100, activation="elu")(classifier)

    classifier = Dense(units=num_classes, activation="softmax")(classifier)

    model_final = Model(autoencoder.input, classifier)
    model_final.compile(loss="categorical_crossentropy", 
                        optimizer=keras.optimizers.Adam(), 
                        metrics=[metrics.categorical_accuracy])

    model_final.summary()

    model_final.fit(X_train, keras.utils.to_categorical(y_train, num_classes),
                    validation_split=0.1,
                    epochs=50,
                    batch_size=100,
                    shuffle=True,
                    verbose=1)

print(roc_auc_score(lb.transform(y_test.reshape(y_test.shape[0], 1)), 
                    model_final.predict(X_test), 
                    average="weighted"))

print(model_final.evaluate(X_test, keras.utils.to_categorical(y_test, num_classes)))